In [1]:
import os
import glob
import numpy as np
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import gradio as gr
from langchain_core.messages import SystemMessage, HumanMessage
from langchain.llms import HuggingFacePipeline



c:\Users\hitan\anaconda3\envs\torch_llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#loading the llama model by using huggingface pipelines
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

Llama = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(Llama)
model = AutoModelForCausalLM.from_pretrained(
    Llama,
    dtype = torch.float16,
    device_map = "auto"
)

model.config.pad_token_id = model.config.eos_token_id

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,  ####### 512->256
    temperature=0.3,      ####### 0.7->0.3
    do_sample = True,
    top_p = 0.9   
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.43s/it]
Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [3]:
# number of files in the document
knowledge_base_path = "knowledge-base/**/*.md"
files = glob.glob(knowledge_base_path, recursive=True)
print(f"Found {len(files)} files in the knowledge base")

# number of characters in all the documents
entire_knowledge_base = ""

for file_path in files:
    with open(file_path, 'r', encoding="utf8") as f:
        entire_knowledge_base += f.read()
        entire_knowledge_base += "\n\n"
print(f"Total characters in knowledge base: {len(entire_knowledge_base):,}")

Found 76 files in the knowledge base
Total characters in knowledge base: 304,434


In [4]:
# number of tokens in all the documents

tokenizer = AutoTokenizer.from_pretrained(Llama)################################
# encoding = tiktoken.encoding_for_model(Llama)
# tokens = encoding.encode(entire_knowledge_base)
tokens = tokenizer.encode(entire_knowledge_base)######################


token_count = len(tokens)
print(f"Total tokens for {Llama}: {token_count:,}")

Total tokens for meta-llama/Llama-3.2-3B-Instruct: 63,715


In [5]:
# Load in everything in the knowledgebase using LangChain's loaders

folders = glob.glob("knowledge-base/*")

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'})
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

print(f"Loaded {len(documents)} documents")



Loaded 76 documents


In [6]:
documents[1]

Document(metadata={'source': 'knowledge-base\\company\\careers.md', 'doc_type': 'company'}, page_content="# Careers at Insurellm\n\n## Why Join Insurellm?\n\nAt Insurellm, we're not just building software—we're revolutionizing an entire industry. Since our founding in 2015, we've evolved from a high-growth startup to a lean, profitable company with 32 highly talented employees managing 32 active contracts across all eight of our product lines.\n\nAfter reaching 200 employees in 2020, we strategically restructured in 2022-2023 to focus on sustainable growth, operational excellence, and building a world-class remote-first culture. Today, we're a tight-knit team of exceptional professionals who deliver outsized impact through automation, AI, and strategic focus on high-value enterprise clients—from regional insurers to global reinsurance partners.\n\n### Our Culture\n\nWe live by our core values every day:\n- **Innovation First**: We encourage experimentation and creative problem-solving\

In [7]:
# Divide into chunks using the RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=200,separators=["\n## ", "\n### ", "\n\n", "\n", " ", ""]
)####1000->800
chunks = text_splitter.split_documents(documents)

print(f"Divided into {len(chunks)} chunks")
print(f"First chunk:\n\n{chunks[0]}")

Divided into 606 chunks
First chunk:

page_content='# About Insurellm

Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.

The company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm had reached a peak of 200 employees with 12 offices across the US.' metadata={'source': 'knowledge-base\\company\\about.md', 'doc_type': 'company'}


In [8]:
chunks[100]


Document(metadata={'source': 'knowledge-base\\contracts\\Contract with DriveSmart Insurance for Carllm.md', 'doc_type': 'contracts'}, page_content='4. **Instant Quoting Engine:** High-performance quoting:\n   - Sub-3-second quote generation\n   - Real-time rate optimization across 50+ rating factors\n   - Competitive intelligence with market positioning\n   - Dynamic pricing based on demand and capacity\n   - Multi-product bundling discounts (auto + home)\n   - A/B testing capabilities for pricing strategies\n\n5. **Customizable Coverage Plans:** Flexible product configuration:\n   - State-specific coverage options (all 8 operating states)\n   - Usage-based insurance (UBI) programs\n   - Pay-per-mile options\n   - Rideshare and delivery driver coverage\n   - Classic and collector car programs\n   - SR-22 and high-risk driver programs')

### Part 2

In [9]:
# Pick an embedding model

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

db_name = "vector_db"

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 606 documents


In [10]:
#langchain
retriever = vectorstore.as_retriever(search_kwargs={"k":5}) ##### Increased from default 4
llm = HuggingFacePipeline(pipeline=pipe)

C:\Users\hitan\AppData\Local\Temp\ipykernel_22900\2280543987.py:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [11]:
retriever.invoke("Who is Avery?")


[Document(id='c63844a4-d11b-4f19-9a6f-a30a3bfea301', metadata={'doc_type': 'employees', 'source': 'knowledge-base\\employees\\Avery Lancaster.md'}, page_content='- **2010 - 2013**: Business Analyst at Edge Analytics  \n  Prior to joining Innovate, Avery worked as a Business Analyst, focusing on market trends and consumer preferences in the insurance space. This position laid the groundwork for Avery’s future entrepreneurial endeavors.'),
 Document(id='351b8af7-7daf-46d2-bcb3-e7fd1ac6248c', metadata={'source': 'knowledge-base\\employees\\Avery Lancaster.md', 'doc_type': 'employees'}, page_content="## Other HR Notes\n- **Professional Development**: Avery has actively participated in leadership training programs and industry conferences, representing Insurellm and fostering partnerships.  \n- **Diversity & Inclusion Initiatives**: Avery has championed a commitment to diversity in hiring practices, seeing visible improvements in team representation since 2021.  \n- **Work-Life Balance**: F

In [12]:
llm.invoke("Who is Avery?")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'Who is Avery? (Avery is a name that has been used by several people in different contexts)\nAvery is a unisex given name that has been in use since the 19th century. It is derived from the Old English words "aelf" meaning "elf" and "ric" meaning "ruler" or "noble". The name Avery has been used for both males and females, although it is more commonly associated with males.\n\nThere are several notable individuals with the name Avery:\n\n1. Avery Brooks (born 1948), American actor and director, best known for his role as Benjamin Sisko in the TV series Star Trek: Deep Space Nine.\n2. Avery Culkin (born 1991), American actor and member of the Culkin family, known for his roles in films such as Home Alone and Party Monster.\n3. Avery Johnson (born 1965), American basketball player and coach, who played in the NBA and later became the head coach of the Dallas Mavericks.\n4. Avery Jones (born 1994), American football player, who plays as a wide receiver for the Kansas City Chiefs.\n5. Avery

In [13]:
SYSTEM_PROMPT_TEMPLATE = """
{context}
"""

In [14]:
# explicitly handling greeting messages
def is_greeting(message: str) -> bool:
    greetings = {
        "hi", "hello", "hey", "hey there",
        "good morning", "good afternoon", "good evening",
        "hii", "helo", "yo"
    }
    return message.lower().strip() in greetings

def greeting_response() -> str:
    return (
        "Hello 👋\n\n"
        "I’m InsureLLM, your virtual assistant.\n"
        "You can ask me about policies, coverage, benefits, or anything else you need."
    )

In [15]:
# def answer_question(question: str, history):
#     docs = retriever.invoke(question)
#     context = "\n\n".join(doc.page_content for doc in docs)
#     system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
#     response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
#     return response.content
def answer_question(question, history):
    if is_greeting(question):
        return greeting_response()
    
    docs = retriever.invoke(question)
    if not docs:
        return "I couldn't find relevant information about that. Could you rephrase your question?"
    
    context_parts = []
    for i, doc in enumerate(docs, 1):
        doc_type = doc.metadata.get('doc_type', 'unknown')
        content = doc.page_content[:500]  # Limit context per doc
        context_parts.append(f"[{doc_type.upper()}]\n{content}")
    
    # context = "\n\n".join(doc.page_content for doc in docs)
    context = "\n\n---\n\n".join(context_parts)
    
    system_prompt = SYSTEM_PROMPT_TEMPLATE.format(context=context)
    
    try:
        response = llm.invoke([
            SystemMessage(content=system_prompt), 
            HumanMessage(content=question)
        ])
        
        # Extract text from response
        if hasattr(response, 'content'):
            answer = response.content
        else:
            answer = str(response)
    # response = llm.invoke([SystemMessage(content=system_prompt), HumanMessage(content=question)])
    # IMPROVED: Post-processing to catch hallucinations
        if len(answer.strip()) < 20:
            return "I don't have enough information to answer that question. Please ask something specific about Insurellm."
        
        # Remove generic rambling
        if answer.lower().startswith(("i apologize", "i'm sorry", "i cannot")):
            return "I can only answer questions about Insurellm. Could you ask something specific about the company?"
        
        return answer.strip()
    
    except Exception as e:
        return f"Error processing your question: {str(e)}"
    

In [16]:
answer_question("Who is Averi Lancaster?", [])


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"System: \n[EMPLOYEES]\nAvery Lancaster has demonstrated resilience and adaptability throughout her career at Insurellm, positioning the company as a key player in the insurance technology landscape.\n\n---\n\n[EMPLOYEES]\n## Insurellm Career Progression\n- **2015 - Present**: Co-Founder & CEO  \n  Avery Lancaster co-founded Insurellm in 2015 and has since guided the company to its current position as a leading Insurance Tech provider. Avery is known for her innovative leadership strategies and risk management expertise that have catapulted the company into the mainstream insurance market.  \n\n- **2013 - 2015**: Senior Product Manager at Innovate Insurance Solutions  \n  Before launching Insurellm, Avery was a leading\n\n---\n\n[EMPLOYEES]\n# Avery Lancaster\n\n## Summary\n- **Date of Birth**: March 15, 1985\n- **Job Title**: Co-Founder & Chief Executive Officer (CEO)\n- **Location**: San Francisco, California\n- **Current Salary**: $225,000\n\n---\n\n[EMPLOYEES]\n## Other HR Notes\n-

In [17]:
# gr.ChatInterface(answer_question).launch()
demo = gr.ChatInterface(
    answer_question,
    examples=[
        "Who is Avery Lancaster?",
        "What products does Insurellm offer?",
        "How many employees does Insurellm have?",
        "Tell me about the Markellm product",
        "What is the company culture like?"
    ],
    title="InsureLLM - Insurellm Knowledge Assistant",
    description="Ask me anything about Insurellm company, products, employees, or contracts!",
    theme=gr.themes.Soft(),
)

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [18]:
### extra stuff for clearing concepts

In [19]:
# Let's investigate the vectors

collection = vectorstore._collection
count = collection.count()

sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"There are {count:,} vectors with {dimensions:,} dimensions in the vector store")

There are 606 vectors with 384 dimensions in the vector store


### vector visualization


In [20]:
# Prework

result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['doc_type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [21]:
####### 2d
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [ ]:
############ 3d
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
